In [95]:
class Value:
    
    def __init__(self, value, inputs = (), op = "", label = ""):
        self._value = value
        self._predecesors = set(inputs)
        self._op = op
        self._grad = 0
        self._label = label
        self.backward = lambda: None
        
    def __repr__(self) -> str:
        return (f"value of {self._label} = {self._value}; _gradient = {self._grad}")
    
    def __add__(self, other):
        
        # calculate the gradient for addition
        def backward():
            self._grad = out._grad
            other._grad = out._grad
        
        # calculate the output of operation and assign it's local _gradient 
        out =  Value(value = self._value + other._value, inputs=(self, other), op = "+")
        out.backward = backward
        
        #return the value
        return out
    
    def __mul__(self, other):
        
        # calculate the _gradient for multiplication
        def backward():
            self._grad = out._grad * other._value
            other._grad = out._grad * self._value
        
        # calculate the outpout of the operation and assign it it's respective backward function
        out = Value(value = self._value * other._value, inputs=(self, other), op = "*")
        out.backward = backward
        
        # return the output
        return out
    
    def backPropagation(self):
        '''
        Start the backpropagation from this node
        '''
        
        # set the gradient of this node to 1
        self._grad = 1
        
        # build a helper function to create the topological sorting of the graph
        def topologicalSorting(node):
            # create local variable
            L = []
            s = set()
            
            # recursive sorting
            def topo(node:Value):
                if node not in s:
                    s.add(node)
                    for parent in node._predecesors:
                        topo(parent)
                    L.append(node)
            topo(node)
            return L
        
        # call the backward method on each node, starting from the last one
        for node in topologicalSorting(self)[::-1]:
            node.backward()
        

In [96]:
a = Value(2, label="a")
b = Value(-3, label="b")
c = a + b; c._label="c"
d = c * b; d._label = "d"

In [98]:
d.backPropagation()


-3

In [50]:
c.grad

1

In [76]:
revList = [1,2,3]
revList = revList[::-1]
print(revList)

[3, 2, 1]
